In [5]:
import pandas as pd
from sodapy import Socrata
import os

 
TOKEN_APP = "xv66xLFwvXNRjMns6TbebGarR"
 
client = Socrata("www.datos.gov.co", TOKEN_APP)
 
 

In [22]:
socrata_data_identifier = "jbjy-vk9h"
 
Query = """
select
    id_contrato, nombre_entidad, departamento, modalidad_de_contratacion, valor_del_contrato, fecha_de_firma, proveedor_adjudicado
where
    nombre_entidad LIKE 'UNIDAD DE GESTION PENSIONAL Y PARAFISCALES - UGPP'
limit
10000000
"""
 
 
result = client.get(socrata_data_identifier, query = Query)
 
result_df = pd.DataFrame.from_records(result)

In [23]:
print(result_df.dtypes)

id_contrato                  object
nombre_entidad               object
departamento                 object
modalidad_de_contratacion    object
valor_del_contrato           object
fecha_de_firma               object
proveedor_adjudicado         object
dtype: object


In [43]:
result_df['valor_del_contrato'] = result_df ['valor_del_contrato'].astype(float)
result_df['fecha_de_firma'] = pd.to_datetime(result_df['fecha_de_firma'], format='mixed')

In [41]:
print(result_df.dtypes)

id_contrato                   object
nombre_entidad                object
departamento                  object
modalidad_de_contratacion     object
valor_del_contrato           float64
fecha_de_firma                object
proveedor_adjudicado          object
dtype: object


In [32]:
result_df

,id_contrato,nombre_entidad,departamento,modalidad_de_contratacion,valor_del_contrato,fecha_de_firma,proveedor_adjudicado
0,CO1.PCCNTR.2193149,UNIDAD DE GESTION PENSIONAL Y PARAFISCALES - UGPP,Distrito Capital de Bogotá,Contratación directa,41523626.0,2021-02-01,ANA MILENA VARGAS AMAYA
1,CO1.PCCNTR.5982280,UNIDAD DE GESTION PENSIONAL Y PARAFISCALES - UGPP,Distrito Capital de Bogotá,Contratación directa,8927010.0,2024-02-21,MILTON GIOVANNY
2,CO1.PCCNTR.280189,UNIDAD DE GESTION PENSIONAL Y PARAFISCALES - UGPP,Distrito Capital de Bogotá,Contratación directa,8400000.0,2018-01-11,ALEYDA YANETH CARO CASTAÑEDA
3,CO1.PCCNTR.4376829,UNIDAD DE GESTION PENSIONAL Y PARAFISCALES - UGPP,Distrito Capital de Bogotá,Contratación directa,23087052.0,2023-01-06,ALDO JR PATIÑO
4,CO1.PCCNTR.277084,UNIDAD DE GESTION PENSIONAL Y PARAFISCALES - UGPP,Distrito Capital de Bogotá,Contratación directa,16480000.0,2018-01-09,rocio yolenny vargas galindo
...,...,...,...,...,...,...,...
2517,CO1.PCCNTR.2115837,UNIDAD DE GESTION PENSIONAL Y PARAFISCALES - UGPP,Distrito Capital de Bogotá,Contratación directa,335358912.0,2021-01-14,ALBERTO PULIDO RODRIGUEZ S A S
2518,CO1.PCCNTR.271971,UNIDAD DE GESTION PENSIONAL Y PARAFISCALES - UGPP,Distrito Capital de Bogotá,Contratación directa,9600000.0,2018-01-05,YAMILE LARA QUINTERO
2519,CO1.PCCNTR.2168350,UNIDAD DE GESTION PENSIONAL Y PARAFISCALES - UGPP,Distrito Capital de Bogotá,Contratación directa,41050116.0,2021-01-25,ALEJANDRO DAZA VARON
2520,CO1.PCCNTR.4622885,UNIDAD DE GESTION PENSIONAL Y PARAFISCALES - UGPP,Distrito Capital de Bogotá,Contratación directa,31800000.0,2023-02-14,DIEGO ALEXANDER SANCHEZ HIGUERA


In [11]:
print(result_df.columns)

Index(['id_contrato', 'nombre_entidad', 'departamento',
       'modalidad_de_contratacion', 'valor_del_contrato', 'fecha_de_firma'],
      dtype='object')


In [53]:
dataset_contratista = result_df.filter(items=['proveedor_adjudicado', 'valor_del_contrato'])
dataset_contratista['suma_contratos'] = 1
dataset_contratista



,proveedor_adjudicado,valor_del_contrato,suma_contratos
0,ANA MILENA VARGAS AMAYA,41523626.0,1
1,MILTON GIOVANNY,8927010.0,1
2,ALEYDA YANETH CARO CASTAÑEDA,8400000.0,1
3,ALDO JR PATIÑO,23087052.0,1
4,rocio yolenny vargas galindo,16480000.0,1
...,...,...,...
2517,ALBERTO PULIDO RODRIGUEZ S A S,335358912.0,1
2518,YAMILE LARA QUINTERO,9600000.0,1
2519,ALEJANDRO DAZA VARON,41050116.0,1
2520,DIEGO ALEXANDER SANCHEZ HIGUERA,31800000.0,1


In [55]:
contratistas_agrupados = dataset_contratista.groupby('proveedor_adjudicado').agg(
    numero_contratos=('suma_contratos', 'count'),
    monto_total=('valor_del_contrato', 'sum')
    ).reset_index()

In [63]:
contratistas_ordenados = contratistas_agrupados.sort_values(by=['numero_contratos', 'monto_total'], ascending=[False, False])
contratistas_ordenados

,proveedor_adjudicado,numero_contratos,monto_total
902,Sin Descripcion,62,1.294097e+08
862,SERVICIOS DE INFORMACION Y DE VALOR AGREGADO S...,13,7.448055e+09
642,M@ICROTEL SAS,10,5.187573e+08
792,PEDRO IVAN BARRIENTOS GOMEZ,10,1.623981e+08
173,CORUS SYSTEMS & CONSULTING GROUP COLOMBIA SAS,9,1.088250e+10
...,...,...,...
705,MUNICIPIO DE RONDON,1,0.000000e+00
855,SECRETARIA DISTRITAL DE PLANEACION,1,0.000000e+00
857,SENA DIRECCION GENERAL,1,0.000000e+00
885,SUPERINTENDENCIA DE VIGILANCIA Y SEGURIDAD PRI...,1,0.000000e+00


In [57]:
print(contratistas_ordenados)

                                  proveedor_adjudicado  numero_contratos  \
902                                    Sin Descripcion                62   
862  SERVICIOS DE INFORMACION Y DE VALOR AGREGADO S...                13   
642                                      M@ICROTEL SAS                10   
792                        PEDRO IVAN BARRIENTOS GOMEZ                10   
173      CORUS SYSTEMS & CONSULTING GROUP COLOMBIA SAS                 9   
..                                                 ...               ...   
705                                MUNICIPIO DE RONDON                 1   
855                 SECRETARIA DISTRITAL DE PLANEACION                 1   
857                             SENA DIRECCION GENERAL                 1   
885  SUPERINTENDENCIA DE VIGILANCIA Y SEGURIDAD PRI...                 1   
930            UNIÓN TEMPORAL AON  WILLIS CM.002.2020                 1   

      monto_total  
902  1.294097e+08  
862  7.448055e+09  
642  5.187573e+08  
792  1.